In [1]:
# Import dependencies
import os
import pandas as pd
import numpy as np
from sodapy import Socrata

# Import App Token
from config import SODAPY_APPTOKEN

# Use Socrata API to import the dataset
------------
API endpoint: https://information.stpaul.gov/resource/gppb-g9cg.json

In [2]:
# Set up variables for api endpoint and dataset identifier
query_url = 'information.stpaul.gov'
data_id = 'gppb-g9cg'

# Authenticate the request
client = Socrata(query_url, SODAPY_APPTOKEN)

# Returns results as JSON from API / converted to Python list of dictionaries by sodapy
results = client.get(data_id, limit=300000)
''' *****NOTE: Need to sort thru how to do pages because this works for now, but the dataset will only continue to grow.
Will undoubtedly become problematic.  Will work for now though.'''


# Convert to pandas DataFrame
crime_df = pd.DataFrame.from_records(results)
crime_df.head()

,case_number,date,time,code,incident_type,incident,grid,neighborhood_number,neighborhood_name,block,call_disposition,call_disposition_2,count
0,20166678,2020-08-12T23:49:03.000,2020-08-12T23:49:03.000,600,"Theft, Except Auto Theft",Theft,153,17,17 - Capitol River,11X KELLOGG BD E,A,Advised,1
1,20166663,2020-08-12T23:36:54.000,2020-08-12T23:36:54.000,9954,Proactive Police Visit,Proactive Police Visit,57,2,2 - Greater East Side,117X CLARENCE ST,A,Advised,1
2,20166662,2020-08-12T23:35:04.000,2020-08-12T23:35:04.000,9954,Proactive Police Visit,Proactive Police Visit,89,7,7 - Thomas/Dale(Frogtown),52X THOMAS AV,A,Advised,1
3,20166657,2020-08-12T23:29:37.000,2020-08-12T23:29:37.000,9954,Proactive Police Visit,Proactive Police Visit,87,7,7 - Thomas/Dale(Frogtown),UNIVERSITY AV W & OXFORD,A,Advised,1
4,20166654,2020-08-12T23:22:45.000,2020-08-12T23:22:45.000,9954,Proactive Police Visit,Proactive Police Visit,49,6,6 - North End,44X MARYLAND AV W,A,Advised,1



# Exploratory Analysis

#### TIME

In [3]:
# Find oldest and most recent date
# 'date' column is an object, change it to datetime
crime_df['date'] = pd.to_datetime(crime_df['date'])

# Preview
crime_df.head()

,case_number,date,time,code,incident_type,incident,grid,neighborhood_number,neighborhood_name,block,call_disposition,call_disposition_2,count
0,20166678,2020-08-12 23:49:03,2020-08-12T23:49:03.000,600,"Theft, Except Auto Theft",Theft,153,17,17 - Capitol River,11X KELLOGG BD E,A,Advised,1
1,20166663,2020-08-12 23:36:54,2020-08-12T23:36:54.000,9954,Proactive Police Visit,Proactive Police Visit,57,2,2 - Greater East Side,117X CLARENCE ST,A,Advised,1
2,20166662,2020-08-12 23:35:04,2020-08-12T23:35:04.000,9954,Proactive Police Visit,Proactive Police Visit,89,7,7 - Thomas/Dale(Frogtown),52X THOMAS AV,A,Advised,1
3,20166657,2020-08-12 23:29:37,2020-08-12T23:29:37.000,9954,Proactive Police Visit,Proactive Police Visit,87,7,7 - Thomas/Dale(Frogtown),UNIVERSITY AV W & OXFORD,A,Advised,1
4,20166654,2020-08-12 23:22:45,2020-08-12T23:22:45.000,9954,Proactive Police Visit,Proactive Police Visit,49,6,6 - North End,44X MARYLAND AV W,A,Advised,1


In [6]:
# Preview the tail
'''Notice difference in date and time columns over the years'''
crime_df.tail()

,case_number,date,time,code,incident_type,incident,grid,neighborhood_number,neighborhood_name,block,call_disposition,call_disposition_2,count
272385,14172159,2014-08-14,22:13,640,"Theft, From Auto",Theft,38,2,2 - Greater East Side,FLANDRAU ST & MARYLAND,A,Advised,1
272386,14172444,2014-08-14,23:00,710,"Motor Vehicle Theft, Automobile",Auto Theft,55,5,5 - Payne/Phalen,97X MENDOTA ST,RR,Report Written,1
272387,14172229,2014-08-14,23:56,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,96,4,4 - Dayton's Bluff,BEECH ST & EARL,RR,Report Written,1
272388,14172228,2014-08-14,23:58,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,34,5,5 - Payne/Phalen,IVY AV E & SEARLE,RR,Report Written,1
272389,14171560,2014-08-14,0:00,641,"Theft, From Auto, Under $500",Theft,33,5,5 - Payne/Phalen,132X WESTMINSTER ST,RR,Report Written,1


In [5]:
# Datset start and end dates
recent_date = crime_df.iloc[0][1]
print(f"The earliest date recorded in the dataset: {recent_date}")
first_date = crime_df.iloc[-1][1]
print(f"The earliest date recorded in the dataset: {first_date}")

The earliest date recorded in the dataset: 2020-08-12 23:49:03
The earliest date recorded in the dataset: 2014-08-14 00:00:00


#### TYPES OF CALLS

In [7]:
# Look at types of Call Disposition
crime_df['call_disposition'].unique()

array(['A', 'RR', 'R', 'G'], dtype=object)

In [8]:
# Combine 'RR' and 'R' because they both stand for Report Written from what I can tell from the official dataset
crime_df = crime_df.replace({'R': 'RR'})

# Check unique codes again
crime_df['call_disposition'].unique()

array(['A', 'RR', 'G'], dtype=object)

In [18]:
# Look at 'incident_type' and 'incident' to see if there are similar values
print("INCIDENT TYPE VALUE COUNTS:")
print(crime_df['incident_type'].value_counts())
print("----------")
print("INCIDENT VALUE COUNTS:")
print(crime_df['incident'].value_counts())

INCIDENT TYPE VALUE COUNTS:
Proactive Police Visit                               121795
Theft, Except Auto Theft                              17715
Motor Vehicle Theft, Automobile                       10429
Narcotics                                              9769
Criminal Damage to Property                            9246
                                                      ...  
Theft, From Coin Operated Device, $501 to $1000           1
Murder, Manslaughter By Negligence                        1
Graffiti-Gang (Felony, Over $500)                         1
Arson, Residential, Multiple, Abandoned, Other            1
Robbery, Service Station, Other Dangerous Weapons         1
Name: incident_type, Length: 175, dtype: int64
----------
INCIDENT VALUE COUNTS:
Proactive Police Visit        121795
Theft                          56837
Vandalism                      17801
Auto Theft                     15578
Narcotics                      14569
Burglary                       13683
Simple As

In [19]:
# Combine 'Simple Assault Dom.' (one is misspelled)
crime_df = crime_df.replace({"Simple Asasult Dom.": "Simple Assault Dom."})

# Check value counts again
crime_df['incident'].value_counts()

Proactive Police Visit        121795
Theft                          56837
Vandalism                      17801
Auto Theft                     15578
Narcotics                      14569
Burglary                       13683
Simple Assault Dom.             6996
Discharge                       6668
Community Engagement Event      6300
Robbery                         4168
Agg. Assault                    2847
Agg. Assault Dom.               2518
Graffiti                        1031
Rape                             928
Arson                            643
Homicide                          26
Other                              2
Name: incident, dtype: int64

In [20]:
crime_df

,case_number,date,time,code,incident_type,incident,grid,neighborhood_number,neighborhood_name,block,call_disposition,call_disposition_2,count
0,20166678,2020-08-12 23:49:03,2020-08-12T23:49:03.000,600,"Theft, Except Auto Theft",Theft,153,17,17 - Capitol River,11X KELLOGG BD E,A,Advised,1
1,20166663,2020-08-12 23:36:54,2020-08-12T23:36:54.000,9954,Proactive Police Visit,Proactive Police Visit,57,2,2 - Greater East Side,117X CLARENCE ST,A,Advised,1
2,20166662,2020-08-12 23:35:04,2020-08-12T23:35:04.000,9954,Proactive Police Visit,Proactive Police Visit,89,7,7 - Thomas/Dale(Frogtown),52X THOMAS AV,A,Advised,1
3,20166657,2020-08-12 23:29:37,2020-08-12T23:29:37.000,9954,Proactive Police Visit,Proactive Police Visit,87,7,7 - Thomas/Dale(Frogtown),UNIVERSITY AV W & OXFORD,A,Advised,1
4,20166654,2020-08-12 23:22:45,2020-08-12T23:22:45.000,9954,Proactive Police Visit,Proactive Police Visit,49,6,6 - North End,44X MARYLAND AV W,A,Advised,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272385,14172159,2014-08-14 00:00:00,22:13,640,"Theft, From Auto",Theft,38,2,2 - Greater East Side,FLANDRAU ST & MARYLAND,A,Advised,1
272386,14172444,2014-08-14 00:00:00,23:00,710,"Motor Vehicle Theft, Automobile",Auto Theft,55,5,5 - Payne/Phalen,97X MENDOTA ST,RR,Report Written,1
272387,14172229,2014-08-14 00:00:00,23:56,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,96,4,4 - Dayton's Bluff,BEECH ST & EARL,RR,Report Written,1
272388,14172228,2014-08-14 00:00:00,23:58,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,34,5,5 - Payne/Phalen,IVY AV E & SEARLE,RR,Report Written,1
